In [1]:
import pandas as pd
import numpy as np
import pymssql  
import requests as re
import json
import geopandas as gpd
from shapely import geometry
from matplotlib import pyplot as plt
import os
import requests as re
from scipy.sparse import coo_matrix
import step_3 as stp3
import webbrowser
from selenium import webdriver
# browser = webdriver.Chrome('E:\chromedriver.exe')
import obtain_coord as ocd
# browser1 = webdriver.Chrome('E:\chromedriver.exe')


#  1. construction interdependent edge

In [2]:
bss=gpd.read_file(r'shapefile\bus_station.shp',encoding='utf-8') 
mss=gpd.read_file(r'shapefile\subway_station.shp',encoding='utf-8') 

In [3]:
bss=gpd.read_file(r'shapefile\bus_station.shp',encoding='utf-8') 
mss=gpd.read_file(r'shapefile\subway_station.shp',encoding='utf-8')

R=1200
mssf=mss.copy()
mssf=mssf.to_crs('+init=epsg:3395')
ge=mssf.buffer(R)
m_bf=gpd.GeoDataFrame(geometry=ge) 
m_bf.index=list(mss['station'])

bssf=bss.copy()
bssf=bssf.to_crs('+init=epsg:3395')

mbm = gpd.sjoin(bssf,m_bf,predicate='within')
mbm.head()

,station,lng,lat,geometry,index_right
0,霍营北口,116.36554,40.08844,POINT (12953752.659 4851280.998),回龙观东大街
129,上坡佳园西门,116.36671,40.08310,POINT (12953882.902 4850507.080),回龙观东大街
1065,龙锦三街东口,116.36533,40.08420,POINT (12953729.282 4850666.496),回龙观东大街
1440,天鑫家园,116.37534,40.08203,POINT (12954843.590 4850352.014),回龙观东大街
1792,华龙苑北里北门,116.37045,40.08166,POINT (12954299.237 4850298.394),回龙观东大街


In [4]:
def spyder_walk(AK,origin,destination):
    url='https://restapi.amap.com/v5/direction/walking?parameters'
    parameters={'key':AK,
               'isindoor':'0',
               'origin':origin,
               'destination':destination,
               'output':'json'}
    r = re.get(url,parameters)
    r_js = r.json()
    return r_js
def convert_coord(AK,org):
    url='https://restapi.amap.com/v3/assistant/coordinate/convert?parameters'
    parameters={'key':AK,
                'locations':org,
                'coordsys':'gps',
                'output':'json'}
    r = re.get(url,parameters)
    r_js = r.json()
    return r_js

In [ ]:
# 
AK='fcddbc8c98225666980450a6296f2cc8'
sbw=list(mss['station'])
lt_t=[]
lt_hc=[]
for i in range(len(sbw)):
    org=str(round(float(mss.iloc[i,1]),5))+','+str(round(float(mss.iloc[i,2]),5))
    r_js=convert_coord(AK,org)
    origin=str(round(float(r_js['locations'].split(',')[0]),5))+','+str(round(float(r_js['locations'].split(',')[1]),5))
    mbm_i=list(mbm[mbm['index_right']==sbw[i]]['station'])
    time_all=[]
    hc_b=[]
    if len(mbm_i)!=0:
        for j in mbm_i:
            dst=str(round(float(list(bss[bss['station']==j]['lng'])[0]),5))+','+str(round(float(list(bss[bss['station']==j]['lat'])[0]),5))
            r_js=convert_coord(AK,dst) # 
            destination=str(round(float(r_js['locations'].split(',')[0]),5))+','+str(round(float(r_js['locations'].split(',')[1]),5))
            r_js=spyder_walk(AK,origin,destination)
            time_all.append(float(r_js['route']['paths'][0]['distance'])/83)
            # 83：
            hc_b.append(j)
    else:
        time_all=-1
        hc_b=-1
    lt_t.append(time_all)
    lt_hc.append(hc_b)
sbtobs=pd.DataFrame()
sbtobs['station']=sbw
sbtobs['tostations']=lt_hc
sbtobs['to_time']=lt_t

In [ ]:
# 
for i in range(sbtobs.shape[0]):
    if sbtobs.iloc[i,1]!=-1:
        if len(sbtobs.iloc[i,1])>=2:
            si=sbtobs.iloc[i,1]
            st=sbtobs.iloc[i,2]
            si_idi=[si[c] for c in range(len(st)) if st[c]<=10]
            st_idi=[st[c] for c in range(len(st)) if st[c]<=10]
            if len(si_idi)==0:
                st_idi=[min(st)]
                si_idi=[si[int(st.index(min(st)))]]
            sbtobs.loc[i]=[sbtobs.iloc[i,0],si_idi,st_idi]

In [7]:
sbtobs=sbtobs[sbtobs['tostations']!=-1]

In [ ]:
sbtobs=sbtobs.reset_index()
sbtobs=sbtobs.drop(columns='index')

In [9]:
for i in range(sbtobs.shape[0]):
    sti1=sbtobs.iloc[i,0]+'_m'
    stis=sbtobs.iloc[i,1]
    stis=[c+'_b' for c in stis]
    sbtobs.loc[i]=[sti1,stis,sbtobs.iloc[i,2]]

C:\Users\111\AppData\Roaming\Python\Python37\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [11]:
sbtobs.shape

(277, 3)

In [3]:
def split_x(x):
    return x.replace('[','').replace(']','').replace("'",'').replace(' ','').split(',')
def split_t(x):
    t=x
    t = t.replace('[', '').replace(']', '').replace(' ', '').replace("'", '').split(',')
    t = [float(c) for c in t]
    return t
sbtobs=pd.read_csv('公交换乘地铁的步行时间.csv',encoding='gbk')
sbtobs['tostations']=sbtobs['tostations'].apply(lambda x:split_x(x))
sbtobs['to_time']=sbtobs['to_time'].apply(lambda x:split_t(x))
sbtobs=sbtobs.drop(columns=['Unnamed: 0'])

#  2. adjacent matrix in BRTN

In [4]:
bsl=gpd.read_file(r'shapefile\bus_line.shp',encoding='utf-8') 
mtl=gpd.read_file(r'shapefile\subway_line.shp',encoding='utf-8') 

def split_m(x):
    x=x.replace('[','').replace(']','').replace(' ','').replace("'",'').split(',')
    x=[c+'_m' for c in x]
    return x
mtl['stations']=mtl['stations'].apply(lambda x:split_m(x))

def split_b(x):
    x=x.replace('[','').replace(']','').replace(' ','').replace("'",'').split(',')
    x=[c+'_b' for c in x]
    return x
bsl['stations']=bsl['stations'].apply(lambda x:split_b(x))
bsl.head(1)

,line,stations,geometry
0,43,"[五间楼_b, 贾家花园_b, 赵公口桥南_b, 刘家窑桥西_b, 刘家窑桥北_b, 蒲黄榆...","LINESTRING (116.42366 39.85051, 116.41722 39.8..."


In [6]:
sbs=list(bss['station'])
sms=list(mss['station'])
sbs=[c+'_b' for c in sbs]
sms=[c+'_m' for c in sms]
ss_all=sbs.copy()
ss_all.extend(sms)
mb_adj=pd.DataFrame(np.zeros([len(ss_all),len(ss_all)]),columns=ss_all)
mb_adj.index=ss_all
mb_adj.head()

,霍营北口_b,新苑街_b,建功北里_b,万盛北里_b,莲怡园北路_b,百望山森林公园_b,焦王庄北口_b,小马厂_b,广阳大街_b,天通东苑二区_b,...,北土城_m,陶然桥_m,郭公庄_m,北京西站_m,马家堡_m,天通苑南_m,临河里_m,张郭庄_m,永定门外_m,化工_m
霍营北口_b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
新苑街_b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
建功北里_b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
万盛北里_b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
莲怡园北路_b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
sbtobs[sbtobs['station']=='天通苑南']

,station,tostations,to_time


In [9]:
# 构建地铁内部的连接边
for i in range(mtl.shape[0]):
    stis=mtl.iloc[i,1]
    for j in range(1,len(stis)):
        mb_adj.loc[stis[j-1]][stis[j]]=1
# 构建公交内部的连接边
for j in range(bsl.shape[0]):
    stjs=bsl.iloc[j,1]
    for k in range(1,len(stjs)):
        mb_adj.loc[stjs[k-1]][stjs[k]]=1
# # 构建地铁-公交的连接边
for j in range(sbtobs.shape[0]):
    stj=sbtobs.iloc[j,0]
    stjs=sbtobs.iloc[j,1]
    for k in stjs:
        mb_adj.loc[stj][k]=1
mb_adj.head()

,霍营北口_b,新苑街_b,建功北里_b,万盛北里_b,莲怡园北路_b,百望山森林公园_b,焦王庄北口_b,小马厂_b,广阳大街_b,天通东苑二区_b,...,北土城_m,陶然桥_m,郭公庄_m,北京西站_m,马家堡_m,天通苑南_m,临河里_m,张郭庄_m,永定门外_m,化工_m
霍营北口_b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
新苑街_b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
建功北里_b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
万盛北里_b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
莲怡园北路_b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 3.travel time and wark time of each OD

In [12]:
import spyder_time as spt

In [ ]:

D=mb_adj.values
c=coo_matrix(D)
sy=list(mb_adj.index)
org_all=[]
dst_all=[]
time_all=[]

icd=pd.read_feather(r'F:\201909公交刷卡数据\IC数据\LK20190910.feather')
icd_b=icd[(icd['f_mode']=='\tGJ')&(icd['t_mode']=='#GJ')]
icd_m=icd[(icd['f_mode']=='\tDT')&(icd['t_mode']=='#DT')]
df_time=pd.DataFrame()
for i in range(c.row.shape[0]):
    org=sy[int(c.row[i])]
    dst=sy[int(c.col[i])]
    org_all.append(org)
    dst_all.append(dst)
    # 分析od是否为同种交通方式站点
    org_i=org.split('_')[0]
    dst_i=dst.split('_')[0]
    if org[-1]==dst[-1]:
        if dst[-1]=='b':
            bt1 = icd_b[(icd_b['fst_name'] == org_i) & (icd_b['tst_name'] == dst_i)]
            if bt1.shape[0]!=0:
                t_all = []
                fts = list(bt1['f_tm'])
                tts = list(bt1['t_tm'])
                for k in range(len(fts)):
                    t_all.append((datetime.datetime.strptime(tts[k], "%Y%m%d%H%M%S") - datetime.datetime.strptime(
                        fts[k], "%Y%m%d%H%M%S")).total_seconds() / 60)
                time_od=min(t_all)
            else:
                time_od=-1
        else:
            mt1 = icd_m[(icd_m['fst_name'] == org_i) & (icd_m['tst_name'] == dst_i)]
            if mt1.shape[0]!=0:
                mt_all = []
                mfts = list(mt1['f_tm'])
                mtts = list(mt1['t_tm'])
                for k in range(len(mfts)):
                    mt_all.append((datetime.datetime.strptime(mtts[k], "%Y%m%d%H%M%S") - datetime.datetime.strptime(
                        mfts[k], "%Y%m%d%H%M%S")).total_seconds() / 60)
                time_od = min(t_all)
            else:
                time_od = -1
    else:
        if org[-1]=='b':
            time_od=list(sbtobs.iloc[sbtobs[sbtobs['station']==dst].index,2])[0][list(
                sbtobs.iloc[sbtobs[sbtobs['station']==org].index,1])[0].index(org)]
        else:
            time_od=list(sbtobs.iloc[sbtobs[sbtobs['station']==org].index,2])[0][list(
                sbtobs.iloc[sbtobs[sbtobs['station']==org].index,1])[0].index(dst)]
    time_all.append(time_od)
df_time['org'] = org_all
df_time['dst'] = dst_all
df_time['time'] = time_all
print(df_time.shape)
print(df_time[df_time['time']==-1].shape)

errs=df_time[df_time['time']==-1]
AK='fcddbc8c98225666980450a6296f2cc8'
for i in range(errs.shape[0]):
    org=errs.iloc[i,0]
    dst=errs.iloc[i,1]
    # 分析od是否为同种交通方式站点
    org_i=org.split('_')[0]
    dst_i=dst.split('_')[0]
    if dst[-1]=='b':
        org_gps=org_i+'公交站'
        org_ngps=spt.spyder_time.convert_coord(AK,org_gps,bss,mss,browser1)
        dst_gps=dst_i+'公交站'
        dst_ngps=spt.spyder_time.convert_coord(AK,dst_gps,bss,mss,browser1)
        time_od=spt.spyder_time.bus_time(AK,org_ngps,dst_ngps)
    else:
        org_gps=org_i+'地铁站'
        org_ngps=spt.spyder_time.convert_coord(AK,org_gps,bss,mss,browser1)
        dst_gps=dst_i+'地铁站'
        dst_ngps=spt.spyder_time.convert_coord(AK,dst_gps,bss,mss,browser1)
        time_od=spt.spyder_time.sub_time(AK,org_ngps,dst_ngps,dst_gps,org_gps)
    errs.loc[i]=[org,dst,time_od]
df_time=df_time[df_time['time']!=-1]
df_time=pd.concat([df_time,errs])
print(df_time[df_time['time']==-1].shape)

##  3.2 bi-direction adajcent matrix
      时间加权
      无向无权

In [10]:

D=mb_adj.values
DW=np.zeros(D.shape)
sy=list(mb_adj.index)
print(sum(sum(D)))
print(np.max(D))
print('      ')
for i in range(D.shape[0]):
    for j in range(D.shape[1]):
        if j==i:
            D[i,j]=0
print(sum(sum(D)))
print(np.max(D))
print('      ')
for i in range(D.shape[0]):
    for j in range(D.shape[1]):
        if D[i,j]==1:
            D[j,i]=1
            org=sy[i]
            dst=sy[j]
            try:
                DW[i,j]=df_time[(df_time['org']==org)&(df_time['dst']==dst)]['time'].to_list()[0]
                DW[j,i]=df_time[(df_time['org']==org)&(df_time['dst']==dst)]['time'].to_list()[0]
            except:
                DW[i,j]=df_time[(df_time['org']==dst)&(df_time['dst']==org)]['time'].to_list()[0]
                DW[j,i]=df_time[(df_time['org']==dst)&(df_time['dst']==org)]['time'].to_list()[0]
print(sum(sum(D)))
print(np.max(D))
print('      ')
mb_adj1=pd.DataFrame(D,columns=mb_adj.columns)
mb_adj1.index=mb_adj.index
w_adj=pd.DataFrame(DW,columns=mb_adj.columns)
w_adj.index=mb_adj.index

14.61111111111111

In [11]:
sbtobs.to_csv('公交换乘地铁的步行时间.csv',index=None,encoding='gbk')
df_time.to_csv('公交-地铁路段行程时间.csv',index=None,encoding='gbk')
mb_adj.to_csv('公交-地铁邻接矩阵.csv',encoding='gbk')
w_adj.to_csv('公交-地铁时间加权矩阵.csv',encoding='gbk')